In [177]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [415]:
from fastai.tabular.all import *
from wwf.tab.export import *
from bnbsl import *

from classes import *
from utils import *

In [230]:
SEASON = '21'
set_seed(int(SEASON))

## Download HKJC odds

In [46]:
path_raw = Path('raw_data')
path_data = Path('data')

path_output = Path('output')

In [47]:
session = requests.Session()
r = session.get('http://bet.hkjc.com')
cookies = r.cookies

In [48]:
odds_url = 'https://bet.hkjc.com/football/getJSON.aspx?jsontype=odds_chl.aspx'
response = session.post(
    odds_url,
    headers={'referer':'http://bet.hkjc.com'},
    cookies=cookies
)

In [49]:
with open(path_data/'json'/f'odds_chl-{datetime.now().strftime("%Y-%m-%d-%H-%M-%S")}.txt', 'w') as f:
    f.write(response.text)

In [50]:
matches_json = json.loads(response.text)[1]['matches']

In [51]:
# Example
# matches_json = json.loads(open(path_data/'json'/'odds_chl-2021-09-21-14-03-17.txt').read())[1]['matches']

In [416]:
matches = [Match(m) for m in matches_json if m['matchStatus'] == 'Defined']
odds = [m.export() for m in matches]

['20211107SUN20', '2021-Nov-07', '18:00:00', 'Japanese Division 2 [JD2]', 'Thespakusatsu Gunma', 'SC Sagamihara', 'true', '9.5', '1.88', '1.82', 'false', '10.5', '2.40', '1.50', 'false', '12.5', '4.40', '1.16']
['20211107SUN21', '2021-Nov-07', '18:00:00', 'Korean Division 1 [KD1]', 'FC Seoul', 'Seongnam FC', 'false', '11.5', '4.40', '1.16', 'false', '9.5', '2.60', '1.43', 'true', '8.5', '1.98', '1.73']
['20211107SUN22', '2021-Nov-07', '19:00:00', 'Russian Premier [RPL]', 'Ural Yekaterinburg', 'Zenit St. Petersburg', 'true', '9.5', '1.85', '1.85', 'false', '10.5', '2.35', '1.52', 'false', '13.5', '4.90', '1.13']
['20211107SUN25', '2021-Nov-07', '19:30:00', 'Italian Division 1 [ISA]', 'Venezia', 'Roma', 'false', '10.5', '2.50', '1.46', 'true', '9.5', '1.93', '1.77', 'false', '12.5', '4.15', '1.18']
['20211107SUN26', '2021-Nov-07', '20:00:00', 'French Division 1 [FFL]', 'Marseille', 'Metz', 'false', '13.5', '5.10', '1.12', 'true', '9.5', '1.82', '1.88', 'false', '10.5', '2.25', '1.57']
['

In [417]:
cols_match = ['MatchDay', 'Date', 'Time', 'LeagueJC', 'HomeTeamJC', 'AwayTeamJC']
cols_odds0 = ['MAINLINE_0', 'CHL_LINE_0', 'CHL_H_0', 'CHL_L_0']
cols_odds1 = ['MAINLINE_1', 'CHL_LINE_1', 'CHL_H_1', 'CHL_L_1']
cols_odds2 = ['MAINLINE_2', 'CHL_LINE_2', 'CHL_H_2', 'CHL_L_2']
cols_odds  = ['MAINLINE', 'CHL_LINE', 'CHL_H', 'CHL_L']
cols_pred  = ['alpha_1', 'alpha_2', 'mu_1', 'mu_2', 'omega', 'corr']

cols = cols_match + cols_odds0 + cols_odds1 + cols_odds2

In [418]:
odds = pd.DataFrame(odds, columns=cols)
odds = odds.fillna(value=np.nan)

In [419]:
cols_odds_ = cols_odds0[1:]+cols_odds1[1:]+cols_odds2[1:]
odds[cols_odds_] = odds[cols_odds_].astype(float)

In [420]:
odds['MatchDay'] = odds['MatchDay'].str[8:]
odds['Date'] = pd.to_datetime(odds['Date'])
odds['Time'] = pd.to_datetime(odds['Time'], format='%H:%M:%S').dt.time

In [421]:
odds['DateTimeJC'] = pd.to_datetime(odds['Date'].dt.date.map(str) + '-' + odds['Time'].map(str))
odds['DateTimeJC'] = odds['DateTimeJC'].dt.tz_localize('Hongkong')
odds['DateTime'] = odds['DateTimeJC'].dt.tz_convert('GB')

In [422]:
map_league = pd.read_csv(path_data/'league.csv')
map_team = pd.read_csv(path_data/'team.csv')

In [423]:
# Map Div name
odds = odds.merge(map_league[['LeagueJC', 'Div']], 'inner', on='LeagueJC')

In [424]:
# Map Team name
odds = odds.merge(map_team[['TeamNameJC', 'TeamName']].rename(columns={'TeamName':'HomeTeam'}), 'inner', 
                  left_on='HomeTeamJC', right_on='TeamNameJC').drop(columns=['TeamNameJC'])
odds = odds.merge(map_team[['TeamNameJC', 'TeamName']].rename(columns={'TeamName':'AwayTeam'}), 'inner', 
                  left_on='AwayTeamJC', right_on='TeamNameJC').drop(columns=['TeamNameJC'])

## Download recent stats

In [425]:
# Download latest results in current season
!wget -q https://www.football-data.co.uk/mmz4281/{SEASON}{int(SEASON)+1}/data.zip -O raw_data/data.zip

# Unzip to folder
!unzip -q -o raw_data/data.zip -d raw_data/{SEASON}

In [426]:
usecols = ['Div', 'Date', 'HomeTeam', 'AwayTeam', 'HC', 'AC', 'FTHG', 'FTAG', 'HS', 'AS', 'HST', 'AST']
dtype = {'HC':'float', 'AC':'float'}
parse_dates = ['Date']

seasons = [SEASON]

dfs = []

for folder in sorted(path_raw.iterdir()):
    if folder.is_dir() and folder.name in seasons: 
        for file in sorted(folder.glob('*.csv')):
            try:
                df = pd.read_csv(file, usecols=usecols, dtype=dtype, parse_dates=parse_dates, dayfirst=True)
                df['Season'] = folder.name
                dfs.append(df)
            except:
                continue

In [427]:
df_season = pd.concat(dfs)
df_season = df_season.dropna()
df_season = df_season.sort_values(['Div', 'Date', 'HomeTeam']).reset_index(drop=True)

In [428]:
df_hist = pd.read_csv(path_data/'data.csv', dtype={'HC':'float', 'AC':'float'}, parse_dates=['Date'])
df_hist = df_hist.query(f'Season == {int(SEASON)-1}').reset_index(drop=True)

In [429]:
df_season = pd.concat([df_hist[df_season.columns], df_season])

In [430]:
# Make features on historical stats (Home and Away)
stats = ['FTHG', 'HS', 'HST', 'HC', 'FTAG', 'AS', 'AST', 'AC']
df_home, df_away = joinLastGamesStatsHomeAway(df_season, stats)

In [431]:
# Make features on historical stats (For and Against)
stats = [('FTHG', 'FTAG', 'FTG'), ('HS', 'AS', 'S'), ('HST', 'AST', 'ST'), ('HC', 'AC', 'C')]
df_for, df_against = joinLastGamesStatsForAgainst(df_season, stats)

In [432]:
df_home = df_home.sort_values(['HomeTeam', 'Date']).reset_index(drop=True)
df_away = df_away.sort_values(['AwayTeam', 'Date']).reset_index(drop=True)

df_home = df_home.groupby('HomeTeam')[df_home.columns[df_home.columns.str.contains('Avg')]].last().reset_index()
df_away = df_away.groupby('AwayTeam')[df_away.columns[df_away.columns.str.contains('Avg')]].last().reset_index()

In [433]:
odds = odds.merge(df_home, 'left', 'HomeTeam').merge(df_away, 'left', 'AwayTeam')

In [434]:
cols_home = df_for.columns[df_for.columns.str.contains('Avg')]
cols_home = dict(zip(cols_home, 'Home'+cols_home))
cols_home.update({'Team':'HomeTeam'})

cols_away = df_for.columns[df_for.columns.str.contains('Avg')]
cols_away = dict(zip(cols_home, 'Away'+cols_away))
cols_away.update({'Team':'AwayTeam'})

df_for = df_for.groupby('Team')[df_for.columns[df_for.columns.str.contains('Avg')]].last().reset_index()
odds = odds.merge(df_for.rename(columns=cols_home), 'left', 'HomeTeam').merge(df_for.rename(columns=cols_away), 'left', 'AwayTeam')

In [435]:
cols_home = df_against.columns[df_against.columns.str.contains('Avg')]
cols_home = dict(zip(cols_home, 'Home'+cols_home))
cols_home.update({'Team':'HomeTeam'})

cols_away = df_against.columns[df_against.columns.str.contains('Avg')]
cols_away = dict(zip(cols_home, 'Away'+cols_away))
cols_away.update({'Team':'AwayTeam'})

df_against = df_against.groupby('Team')[df_against.columns[df_against.columns.str.contains('Avg')]].last().reset_index()
odds = odds.merge(df_against.rename(columns=cols_home), 'left', 'HomeTeam').merge(df_against.rename(columns=cols_away), 'left', 'AwayTeam')

In [436]:
add_datepart(odds, 'DateTime', prefix='', drop=False);

In [437]:
display_df(odds.head(5).T)

,0,1,2,3,4
MatchDay,SUN25,SUN41,SUN42,SUN71,SUN72
Date,2021-11-07 00:00:00,2021-11-07 00:00:00,2021-11-07 00:00:00,2021-11-08 00:00:00,2021-11-08 00:00:00
Time,19:30:00,22:00:00,22:00:00,01:00:00,01:00:00
LeagueJC,Italian Division 1 [ISA],Italian Division 1 [ISA],Italian Division 1 [ISA],Italian Division 1 [ISA],Italian Division 1 [ISA]
HomeTeamJC,Venezia,Sampdoria,Udinese,Napoli,Lazio
AwayTeamJC,Roma,Bologna,Sassuolo,Verona,Salernitana
MAINLINE_0,false,false,true,true,false
CHL_LINE_0,10.5,13.5,10.5,9.5,13.5
CHL_H_0,2.5,4.15,2.17,1.87,4.75
CHL_L_0,1.46,1.18,1.61,1.83,1.14


## Load model

In [438]:
learn_bnb = load_learner('models/learn_bnbsl.pkl')

In [439]:
to = load_pandas('models/to.pkl')

In [440]:
def predict(self, row):
    "Predict on a Pandas Series"
    dl = self.dls.test_dl(row.to_frame().T)
    dl.dataset.conts = dl.dataset.conts.astype(np.float32)
    inp,preds,_ = self.get_preds(dl=dl, with_input=True, with_decoded=False)
    b = tuplify(inp)
    full_dec = self.dls.decode(b)
    return full_dec,preds[0]

learn_bnb.predict = MethodType(predict, learn_bnb)

In [441]:
to_tst = to.new(odds)
to_tst.process()
# to_tst.items.head()

In [442]:
tst_dl = learn_bnb.dls.valid.new(to_tst)
tst_dl.show(max_n=999)

,Div,HomeTeam,AwayTeam,Dayofweek,FTHGLast5Avg,HSLast5Avg,HSTLast5Avg,HCLast5Avg,FTAGLast5Avg,ASLast5Avg,ASTLast5Avg,ACLast5Avg,HomeFTGForLast5Avg,HomeSForLast5Avg,HomeSTForLast5Avg,HomeCForLast5Avg,AwayFTGForLast5Avg,AwaySForLast5Avg,AwaySTForLast5Avg,AwayCForLast5Avg,HomeFTGAgainstLast5Avg,HomeSAgainstLast5Avg,HomeSTAgainstLast5Avg,HomeCAgainstLast5Avg,AwayFTGAgainstLast5Avg,AwaySAgainstLast5Avg,AwaySTAgainstLast5Avg,AwayCAgainstLast5Avg,Year,Month,Week,Day,Dayofyear
0,I1,Venezia,Roma,6,1.2,10.4,3.2,3.8,2.0,16.8,6.8,7.0,1.0,7.6,3.0,3.6,1.2,17.4,4.6,7.4,1.4,12.4,4.4,3.6,1.0,8.8,3.4,2.4,2021.0,11.0,44.0,7.0,311.0
1,I1,Sampdoria,Bologna,6,1.4,11.0,3.2,4.6,1.2,13.2,4.8,5.6,1.8,9.8,2.8,4.0,1.6,14.0,4.8,5.4,2.6,15.2,5.8,5.6,2.4,13.2,5.2,2.4,2021.0,11.0,44.0,7.0,311.0
2,I1,Udinese,Sassuolo,6,1.2,12.0,4.8,3.6,2.0,13.2,6.6,5.6,1.2,13.0,4.4,4.8,1.8,13.2,6.0,4.0,1.4,11.4,4.2,5.0,1.2,14.2,3.6,6.0,2021.0,11.0,44.0,7.0,311.0
3,I1,Napoli,Verona,6,1.6,16.2,4.8,5.8,1.6,8.8,4.4,3.2,1.6,13.8,5.0,3.8,2.8,9.8,5.0,2.8,0.2,9.6,1.8,5.0,1.6,12.8,4.2,4.8,2021.0,11.0,44.0,7.0,311.0
4,I1,Lazio,Salernitana,6,2.8,19.0,8.6,5.2,1.2,11.2,3.8,4.6,1.6,10.0,4.4,3.6,1.2,14.0,5.2,6.4,2.0,13.4,4.6,4.8,1.6,14.6,5.0,5.4,2021.0,11.0,44.0,7.0,311.0
5,I1,Milan,Inter,6,2.2,17.2,4.0,6.2,2.2,14.0,4.6,5.4,2.6,14.6,5.0,3.4,1.6,16.0,4.8,5.2,1.4,11.2,3.8,4.8,1.4,15.8,5.6,6.2,2021.0,11.0,44.0,7.0,311.0
6,F1,Marseille,Metz,6,2.6,15.2,4.6,5.0,1.2,9.0,4.0,4.2,1.4,12.4,3.0,5.0,1.2,6.4,2.8,3.6,1.4,12.4,4.8,5.0,2.6,15.2,5.0,6.8,2021.0,11.0,44.0,7.0,311.0
7,F1,Lorient,Brest,6,1.4,7.2,2.6,2.8,1.0,9.0,3.6,4.8,1.0,11.0,3.6,5.2,1.0,9.8,3.0,4.2,1.4,12.0,3.6,5.6,1.8,13.2,4.8,5.0,2021.0,11.0,44.0,7.0,311.0
8,F1,Nantes,Strasbourg,6,1.4,14.0,5.8,5.4,1.4,8.6,3.0,4.2,1.8,10.8,4.6,4.0,1.6,8.4,3.0,5.6,1.2,9.2,2.6,3.8,1.0,13.2,4.0,6.6,2021.0,11.0,44.0,7.0,311.0
9,F1,Reims,Monaco,6,1.4,11.8,4.6,4.8,1.4,8.4,3.2,3.2,1.2,10.8,3.4,4.8,2.4,11.0,4.0,5.0,1.6,9.4,4.2,4.8,1.0,10.2,3.8,3.2,2021.0,11.0,44.0,7.0,311.0


In [443]:
pred, _ = learn_bnb.get_preds(dl=tst_dl)

In [444]:
alpha = F.softplus(pred[:, 0:2])
mu = F.softplus(pred[:, 2:4])
omega = pred[:, 4]
# omega = torch.tanh(pred[:, 4]) * 50.

total_count = 1. / alpha
logits = torch.log(alpha * mu)

# print(alpha, mu, omega)

In [445]:
# Method 1:
corr = omega * torch.sqrt(mu.prod(-1)) * alpha.prod(-1) / ((1.+alpha)**(1./alpha+1.)).prod(-1) / torch.sqrt((1.+alpha*mu).prod(-1))

In [446]:
# Method 2:
# d = 1.-math.exp(-1)
# corr = omega * d**2 * torch.sqrt(mu.prod(-1)*(1.+alpha*mu).prod(-1)) * ((1.+d*alpha*mu)**(-1-1/alpha)).prod(-1)

In [447]:
odds[cols_pred] = torch.cat([alpha, mu, omega.unsqueeze(-1), corr.unsqueeze(-1)], dim=-1)

In [448]:
odds0 = odds[cols_match+cols_odds0+cols_pred].rename(columns=dict(zip(cols_odds0, cols_odds)))
odds1 = odds[cols_match+cols_odds1+cols_pred].rename(columns=dict(zip(cols_odds1, cols_odds)))
odds2 = odds[cols_match+cols_odds2+cols_pred].rename(columns=dict(zip(cols_odds2, cols_odds)))

In [449]:
odds = pd.concat([odds0, odds1, odds2]).dropna().reset_index(drop=True)
odds['MAINLINE'] = np.where(odds['MAINLINE']=='true', True, False)

In [450]:
odds.head(10)

,MatchDay,Date,Time,LeagueJC,HomeTeamJC,AwayTeamJC,MAINLINE,CHL_LINE,CHL_H,CHL_L,alpha_1,alpha_2,mu_1,mu_2,omega,corr
0,SUN25,2021-11-07,19:30:00,Italian Division 1 [ISA],Venezia,Roma,False,10.5,2.50,1.46,0.236896,0.078677,2.450731,6.644482,-16.824352,-0.095160
1,SUN41,2021-11-07,22:00:00,Italian Division 1 [ISA],Sampdoria,Bologna,False,13.5,4.15,1.18,0.066220,0.035990,4.967203,6.123947,-19.501968,-0.025908
2,SUN42,2021-11-07,22:00:00,Italian Division 1 [ISA],Udinese,Sassuolo,True,10.5,2.17,1.61,0.061318,0.060592,4.843815,5.641300,-18.638985,-0.034983
3,SUN71,2021-11-08,01:00:00,Italian Division 1 [ISA],Napoli,Verona,True,9.5,1.87,1.83,0.078342,0.157631,6.968537,3.020592,-18.360813,-0.083225
4,SUN72,2021-11-08,01:00:00,Italian Division 1 [ISA],Lazio,Salernitana,False,13.5,4.75,1.14,0.054314,0.116225,6.099379,3.780260,-17.734838,-0.048387
5,SUN88,2021-11-08,03:45:00,Italian Division 1 [ISA],AC Milan,Inter Milan,False,13.5,4.75,1.14,0.039407,0.046889,6.019346,6.037274,-21.774935,-0.024968
6,SUN26,2021-11-07,20:00:00,French Division 1 [FFL],Marseille,Metz,False,13.5,5.10,1.12,0.059947,0.112706,6.687933,3.205738,-17.838587,-0.050297
7,SUN43,2021-11-07,22:00:00,French Division 1 [FFL],Lorient,Brest,True,9.5,1.82,1.88,0.059896,0.059590,5.064183,4.417274,-17.141804,-0.028785
8,SUN44,2021-11-07,22:00:00,French Division 1 [FFL],Nantes,Strasbourg,False,10.5,2.35,1.52,0.041924,0.080392,5.574222,3.811283,-16.885044,-0.026338
9,SUN45,2021-11-07,22:00:00,French Division 1 [FFL],Reims,Monaco,False,13.5,5.30,1.11,0.156937,0.161167,2.895845,4.731847,-14.495893,-0.098617


In [451]:
prob_hilo = []

for r in list(zip(odds['alpha_1'], odds['alpha_2'], odds['mu_1'], odds['mu_2'], odds['omega'], odds['CHL_LINE'])):
    total_count = 1. / torch.tensor(r[0:2], device='cpu') 
    logits = torch.log(torch.tensor(r[0:2], device='cpu') * torch.tensor(r[2:4], device='cpu'))
    omega = torch.tensor(r[4], device='cpu')

    bnb_corner = BivariateNegativeBinomialSL(total_count=total_count, omega=omega, logits=logits)
    value = torch.cartesian_prod(torch.arange(0., 15.), torch.arange(0., 15.))
    corner = bnb_corner.log_prob(value).exp()
    
    line = r[5]
    mask = value.sum(-1) < line
    prob_lo = corner[mask].sum()
    prob_hi = 1 - prob_lo
    
    prob_hilo.append([prob_hi.item(), prob_lo.item()])

In [452]:
odds[['prob_hi', 'prob_lo']] = prob_hilo

In [453]:
odds['kelly_hi'] = (odds['prob_hi'] * odds['CHL_H'] - 1) / (odds['CHL_H'] - 1)
odds['kelly_lo'] = (odds['prob_lo'] * odds['CHL_L'] - 1) / (odds['CHL_L'] - 1)

In [454]:
odds['kelly'] = np.where(
    np.maximum(odds['kelly_hi'], odds['kelly_lo']) > 0, 
    np.where(odds['kelly_hi'] > odds['kelly_lo'], odds['kelly_hi'], odds['kelly_lo']), 
    np.nan
)

In [455]:
odds['bet'] = np.where(
    np.maximum(odds['kelly_hi'], odds['kelly_lo']) > 0, 
    np.where(odds['kelly_hi'] > odds['kelly_lo'], 'High', 'Low'), 
    None
)

In [456]:
odds = odds.sort_values('kelly', ascending=False).reset_index(drop=True)

In [457]:
odds['selected'] = np.where(
    odds['MAINLINE']==True, np.where(
        odds['kelly']>0.3, '$$$', np.where(
            odds['kelly']>0.2, '$$', np.where(
                odds['kelly']>0.1, '$', None))), 
    None
)

In [458]:
odds = odds.drop(columns=cols_pred+['kelly_hi', 'kelly_lo'])

In [459]:
odds[odds.bet.notna() & odds.selected.notna()]

,MatchDay,Date,Time,LeagueJC,HomeTeamJC,AwayTeamJC,MAINLINE,CHL_LINE,CHL_H,CHL_L,prob_hi,prob_lo,kelly,bet,selected
2,SUN45,2021-11-07,22:00:00,French Division 1 [FFL],Reims,Monaco,True,9.5,1.85,1.85,0.265063,0.734937,0.423098,Low,$$$
3,SUN88,2021-11-08,03:45:00,Italian Division 1 [ISA],AC Milan,Inter Milan,True,10.5,2.19,1.60,0.636963,0.363037,0.331889,High,$$$
7,SUN33,2021-11-07,21:00:00,Spanish Division 1 [SFL],Villarreal,Getafe,True,8.5,1.85,1.85,0.339574,0.660426,0.260927,Low,$$
8,SUN90,2021-11-08,04:00:00,Spanish Division 1 [SFL],Betis,Sevilla,True,8.5,1.73,1.98,0.671076,0.328924,0.220494,High,$$
9,SUN75,2021-11-08,01:30:00,Spanish Division 1 [SFL],Osasuna,Real Sociedad,True,8.5,1.88,1.82,0.361204,0.638796,0.198303,Low,$
13,SUN31,2021-11-07,20:30:00,German Division 2 [GD2],Schalke 04,Darmstadt,True,9.5,1.73,1.98,0.413387,0.586613,0.164790,Low,$


In [460]:
display_df(odds[odds.MatchDay.isin(odds[odds.bet.notna() & odds.selected.notna()].MatchDay) & odds.bet.notna()])

,MatchDay,Date,Time,LeagueJC,HomeTeamJC,AwayTeamJC,MAINLINE,CHL_LINE,CHL_H,CHL_L,prob_hi,prob_lo,kelly,bet,selected
0,SUN45,2021-11-07,22:00:00,French Division 1 [FFL],Reims,Monaco,False,10.5,2.30,1.54,0.187293,0.812707,0.465867,Low,None
1,SUN45,2021-11-07,22:00:00,French Division 1 [FFL],Reims,Monaco,False,13.5,5.30,1.11,0.054450,0.945550,0.450553,Low,None
2,SUN45,2021-11-07,22:00:00,French Division 1 [FFL],Reims,Monaco,True,9.5,1.85,1.85,0.265063,0.734937,0.423098,Low,$$$
3,SUN88,2021-11-08,03:45:00,Italian Division 1 [ISA],AC Milan,Inter Milan,True,10.5,2.19,1.60,0.636963,0.363037,0.331889,High,$$$
4,SUN33,2021-11-07,21:00:00,Spanish Division 1 [SFL],Villarreal,Getafe,False,12.5,5.55,1.10,0.063130,0.936870,0.305566,Low,None
5,SUN33,2021-11-07,21:00:00,Spanish Division 1 [SFL],Villarreal,Getafe,False,9.5,2.40,1.50,0.238700,0.761300,0.283899,Low,None
6,SUN88,2021-11-08,03:45:00,Italian Division 1 [ISA],AC Milan,Inter Milan,False,11.5,2.80,1.38,0.532539,0.467461,0.272839,High,None
7,SUN33,2021-11-07,21:00:00,Spanish Division 1 [SFL],Villarreal,Getafe,True,8.5,1.85,1.85,0.339574,0.660426,0.260927,Low,$$
8,SUN90,2021-11-08,04:00:00,Spanish Division 1 [SFL],Betis,Sevilla,True,8.5,1.73,1.98,0.671076,0.328924,0.220494,High,$$
9,SUN75,2021-11-08,01:30:00,Spanish Division 1 [SFL],Osasuna,Real Sociedad,True,8.5,1.88,1.82,0.361204,0.638796,0.198303,Low,$


In [461]:
odds.to_csv(path_output/f'odds-{datetime.now().strftime("%Y-%m-%d")}.csv', float_format='%.2f', index=False)

## END